<a href="https://colab.research.google.com/github/yash-jain-1/SarcLM/blob/main/notebooks/LLaVA_Fine_tuning_with_PEFT_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages (run only if not already installed)
%pip install -q -U transformers datasets accelerate peft bitsandbytes trl pyarrow==19.0.0
%pip install -U pip setuptools wheel
%pip install bitsandbytes
%pip install hf_xet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 133.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 83.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [2]:
! pip install -q -U transformers datasets accelerate peft bitsandbytes trl pyarrow==19.0.0

In [3]:
%pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.1 MB/s  0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [ipywidgets]


In [4]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [5]:
# robust_llava_loader.py
# Loads a LLaVA-style model even when AutoModelForCausalLM doesn't recognize LlavaConfig.
# Requires: transformers, torch, datasets, peft, trl, etc.
# Make sure to run: pip install -U "transformers>=4.31.0" bitsandbytes peft trl datasets safetensors
# if you want 4-bit quantization support (and bitsandbytes installed).

import importlib
import sys
import traceback
import random
import torch
import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig  # if bitsandbytes present; import may fail if not installed
from datasets import load_dataset

# CONFIG
BASE_MODEL_NAME = "llava-hf/llava-1.5-7b-hf"
USE_4BIT = False  # set True only if bitsandbytes is installed & you want 4-bit quant
DEVICE_MAP = "auto"
LOW_CPU_MEM = True

def print_versions():
    print("torch:", torch.__version__)
    print("transformers:", transformers.__version__)
    try:
        import bitsandbytes as bnb
        print("bitsandbytes:", bnb.__version__)
    except Exception:
        print("bitsandbytes: NOT INSTALLED")

def try_load_llava_class_and_model(model_name, quant_config=None, device_map="auto", low_cpu_mem=True):
    candidate_module_paths = [
        "transformers.models.llava.modeling_llava",
        "transformers.models.llava.modeling_llava_for_causal_lm",
        "llava.modeling_llava",
        "modeling_llava",
    ]
    candidate_class_names = [
        "LlavaForCausalLM",
        "LlavaModelForCausalLM",
        "LlavaForConditionalGeneration",
        "LlavaModel",
        "LlavaForVision2Seq",
    ]

    last_exc = None
    for mod_path in candidate_module_paths:
        try:
            module = importlib.import_module(mod_path)
        except Exception as e:
            last_exc = e
            continue

        for cls_name in candidate_class_names:
            ModelClass = getattr(module, cls_name, None)
            if ModelClass is None:
                continue

            # Try strategy sequence:
            # 1) If quant_config provided -> try direct (fast path)
            # 2) If ValueError complaining about dispatch -> retry with llm_int8_enable_fp32_cpu_offload + device_map="auto"
            # 3) If still failing -> fallback to no-quant (float16)
            try:
                print(f"Trying to load {cls_name} with device_map={device_map} (quant_config={'yes' if quant_config else 'no'})...")
                return _attempt_from_pretrained(ModelClass, model_name, quant_config, device_map, low_cpu_mem, extra_kwargs={})
            except Exception as e:
                last_exc = e
                tb = traceback.format_exc()
                print(f"Initial attempt with {cls_name} failed: {e}\n{tb}")

                # If message suggests offload, try offload route (only if quant_config not None)
                msg = str(e).lower()
                if quant_config is not None and ("offload" in msg or "dispatched on the cpu" in msg or "some modules are dispatched" in msg):
                    try:
                        print("Retrying with llm_int8_enable_fp32_cpu_offload=True and device_map='auto'...")
                        return ModelClass.from_pretrained(
                            model_name,
                            quantization_config=quant_config,
                            device_map="auto",
                            trust_remote_code=True,
                            low_cpu_mem_usage=low_cpu_mem,
                            llm_int8_enable_fp32_cpu_offload=True,
                        )
                    except Exception as e2:
                        last_exc = e2
                        tb2 = traceback.format_exc()
                        print(f"Retry with offload failed: {e2}\n{tb2}")

                # Final fallback: try without quantization (float16)
                try:
                    print("Retrying without quantization (float16) as a fallback...")
                    return ModelClass.from_pretrained(
                        model_name,
                        device_map=device_map,
                        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                        trust_remote_code=True,
                        low_cpu_mem_usage=low_cpu_mem,
                    )
                except Exception as e3:
                    last_exc = e3
                    tb3 = traceback.format_exc()
                    print(f"Fallback without quantization also failed: {e3}\n{tb3}")
                    continue

    raise RuntimeError("Tried candidate Llava classes but all failed. Last exception:\n" + (str(last_exc) if last_exc is not None else "None"))

def _attempt_from_pretrained(ModelClass, model_name, quant_config, device_map, low_cpu_mem, extra_kwargs):
    """Helper to call from_pretrained with given kwargs and bubble exceptions."""
    try:
        if quant_config is not None:
            return ModelClass.from_pretrained(
                model_name,
                quantization_config=quant_config,
                device_map=device_map,
                trust_remote_code=True,
                low_cpu_mem_usage=low_cpu_mem,
                **extra_kwargs,
            )
        else:
            return ModelClass.from_pretrained(
                model_name,
                device_map=device_map,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                trust_remote_code=True,
                low_cpu_mem_usage=low_cpu_mem,
                **extra_kwargs,
            )
    except Exception as e:
        raise

def load_model_and_tokenizer(model_name, use_4bit=True, device_map="auto", low_cpu_mem=True):
    # Load config + tokenizer
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    quant_config = None
    if use_4bit:
        try:
            quant_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
            )
        except Exception as e:
            print("Could not create BitsAndBytesConfig:", e)
            quant_config = None

    # If LlavaConfig detected: try repo model classes (with fallback paths)
    cfg_name = config.__class__.__name__.lower()
    if "llava" in cfg_name:
        model = try_load_llava_class_and_model(model_name, quant_config=quant_config, device_map=device_map, low_cpu_mem=low_cpu_mem)
    else:
        # generic fallback to AutoModelForCausalLM
        if quant_config is not None:
            try:
                model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    quantization_config=quant_config,
                    device_map=device_map,
                    trust_remote_code=True,
                    low_cpu_mem_usage=low_cpu_mem,
                    llm_int8_enable_fp32_cpu_offload=True,  # safe to include
                )
            except Exception as e:
                print("AutoModelForCausalLM with quant failed, retrying without quant:", e)
                model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    device_map=device_map,
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    trust_remote_code=True,
                    low_cpu_mem_usage=low_cpu_mem,
                )
        else:
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map=device_map,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                trust_remote_code=True,
                low_cpu_mem_usage=low_cpu_mem,
            )

    try:
        model.gradient_checkpointing_enable()
    except Exception:
        pass
    model.config.use_cache = False
    return model, tokenizer

# Now, when you run main(), it will automatically retry with llm_int8_enable_fp32_cpu_offload if needed.

In [7]:
import os
import sys

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Running in Colab. You may use Colab resources.")
    # Optionally: get API key from user input or environment
    # api_key = input("Enter your API key: ")
else:
    print("Not running in Colab. Using local environment.")
    # Optionally: get API key from environment variable
    # api_key = os.getenv("YOUR_API_KEY_ENV_VAR")

Mounted at /content/drive
Running in Colab. You may use Colab resources.


In [ ]:
"""
finetune_after_script_b.py

Uses robust_llava_loader.load_model_and_tokenizer() (script B output) to load the model,
auto-detects good LoRA target_modules, and runs PEFT (LoRA) fine-tuning with trl.SFTTrainer.

Usage: python finetune_after_script_b.py
"""

import os
import re
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel


# try to import helper for preparing k-bit training (peft versions vary)
try:
    from peft import prepare_model_for_kbit_training
except Exception:
    try:
        from peft.utils import prepare_model_for_kbit_training
    except Exception:
        prepare_model_for_kbit_training = None

# Add the src directory to the Python path so we can import robust_llava_loader
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))
from robust_llava_loader import load_model_and_tokenizer

# --------------------- USER CONFIG ---------------------
BASE_MODEL_NAME = "llava-hf/llava-1.5-7b-hf"
DATASET_PATH = "meld_with_rationales.jsonl"   # jsonl containing utterance, sentiment, rationale
OUTPUT_DIR = "./llava-peft-adapters-auto"
USE_4BIT_IF_AVAILABLE = True
MAX_SEQ_LENGTH = 512
PER_DEVICE_BATCH_SIZE = 4
NUM_EPOCHS = 1
LEARNING_RATE = 2e-4
GRADIENT_ACCUMULATION_STEPS = 1
SAVE_STEPS = 200
LOGGING_STEPS = 20
# -------------------------------------------------------

# helper: create the training prompt
def build_prompt(example):
    return (
        "You are a sentiment analysis expert. Analyze the following utterance and provide "
        "the sentiment along with a step-by-step rationale for your decision.\n\n"
        "### Utterance:\n"
        f"{example.get('utterance','')}\n\n"
        "### Analysis:\n"
        f"Sentiment: {example.get('sentiment','')}\n"
        f"Rationale: {example.get('rationale','')}"
    )

# helper: scan model.named_modules() and choose candidate target module name substrings
def auto_detect_target_module_names(model, prefer_text=True):
    """
    Returns a list of module-name substrings to use in LoraConfig.target_modules.
    Strategy:
      - Collect names of submodules that look like projections (q_proj, k_proj, v_proj, out_proj, o_proj)
      - Prefer modules under 'model' that contain tokens like 'self_attn', 'attn', 'q_proj' etc.
      - If prefer_text=True, try to exclude modules under vision tower (module name containing 'vision' or 'vision_tower')
    """
    proj_patterns = set()
    name_list = [n for n, _ in model.named_modules()]

    for n in name_list:
        # skip top-level empty name
        if not n:
            continue
        # skip vision modules if preferring text modules
        if prefer_text and ("vision" in n or "vision_tower" in n or "vision_model" in n):
            continue
        # find typical projection/fc names in module path
        if re.search(r"(q_proj|k_proj|v_proj|o_proj|out_proj|gate_proj|up_proj|down_proj|fc1|fc2)", n):
            # extract final token (last part after '.')
            final = n.split(".")[-1]
            proj_patterns.add(final)
    # fallback if empty
    if not proj_patterns:
        # default common names
        proj_patterns = {"q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"}
    # keep consistent ordering and return as list
    return sorted(list(proj_patterns))

def format_and_map(example, tokenizer):
    text = build_prompt(example)
    eos = tokenizer.eos_token or ""
    return {"text": text + eos}

def main():
    # decide 4-bit usage
    use_4bit = False
    if USE_4BIT_IF_AVAILABLE:
        try:
            import bitsandbytes  # noqa: F401
            use_4bit = True
        except Exception:
            print("bitsandbytes not installed/found — running without 4-bit.")

    # 1) Load model + tokenizer (robust loader)
    print("Loading model + tokenizer (robust loader)...")
    model, tokenizer = load_model_and_tokenizer(model_name=BASE_MODEL_NAME, use_4bit=use_4bit)
    print("Loaded model and tokenizer. Model dtype hint:", getattr(model, "dtype", None))
    model.config.use_cache = False
    try:
        model.gradient_checkpointing_enable()
    except Exception:
        pass

    # 2) Auto-detect target_modules for LoRA (based on model module names)
    print("Auto-detecting candidate LoRA target module name tokens from model.named_modules()...")
    detected = auto_detect_target_module_names(model, prefer_text=True)
    print("Detected target-module name tokens (candidates):", detected)

    # We'll use these tokens as LoraConfig.target_modules (PEFT expects substrings)
    target_modules = detected

    # 3) Prepare model for k-bit training (if using 4-bit & helper present)
    if use_4bit:
        if prepare_model_for_kbit_training is not None:
            print("Preparing model for k-bit training (peft.prepare_model_for_kbit_training)...")
            model = prepare_model_for_kbit_training(model)
        else:
            print("prepare_model_for_kbit_training not available in this peft version — continuing.")

    # 4) Create LoraConfig and wrap model
    lora_cfg = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=target_modules,
    )
    print("Applying LoRA with LoraConfig:", lora_cfg)
    model = get_peft_model(model, lora_cfg)
    print("PEFT/LoRA applied. Peft model keys:", list(model.named_parameters())[:5])

    # 5) Load and format dataset
    print("Loading dataset from", DATASET_PATH)
    ds = load_dataset("json", data_files=DATASET_PATH, split="train")
    print("Dataset size:", len(ds))
    # map to `text` field expected by SFTTrainer
    ds = ds.map(lambda ex: format_and_map(ex, tokenizer), remove_columns=ds.column_names)

    # import here to avoid top-level dependency until needed
    from trl import SFTTrainer, SFTConfig

    # 6) SFTConfig + trainer
    training_args = SFTConfig(
        output_dir=OUTPUT_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        fp16=use_4bit or (torch.cuda.is_available() and torch.cuda.get_device_properties(0).total_memory >= 12 * 1024 ** 2),
        save_steps=SAVE_STEPS,
        logging_steps=LOGGING_STEPS,
        save_total_limit=3,
        report_to="none",
        dataset_text_field="text",
        max_seq_length=MAX_SEQ_LENGTH,
        packing=False,
    )


    trainer = SFTTrainer(
        model=model,
        train_dataset=ds,
        peft_config=lora_cfg,
        tokenizer=tokenizer,
        args=training_args, # Pass the SFTConfig object here
    )

    # 7) Dry-run: single step to validate forward/backward
    print("Running a 1-step dry-run to validate training loop...")
    try:
        trainer.train(max_steps=1)
        print("Dry-run succeeded.")
    except Exception as e:
        print("Dry-run failed — inspect traceback. Error:", e)
        raise

    # 8) Full training
    print("Starting full training...")
    trainer.train()
    print("Training finished.")

    # 9) Save PEFT adapters
    print("Saving adapters to:", OUTPUT_DIR)
    trainer.save_model(OUTPUT_DIR)
    print("Saved. You can load later with PeftModel.from_pretrained(base_model, OUTPUT_DIR)")

if __name__ == "__main__":
    main()


Loading model + tokenizer (robust loader)...
Trying to load LlavaForConditionalGeneration with device_map=auto (quant_config=yes)...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded model and tokenizer. Model dtype hint: torch.float16
Auto-detecting candidate LoRA target module name tokens from model.named_modules()...
Detected target-module name tokens (candidates): ['down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj']
Preparing model for k-bit training (peft.prepare_model_for_kbit_training)...
Applying LoRA with LoraConfig: LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=64, target_modules={'o_proj', 'k_proj', 'up_proj', 'gate_proj', 'q_proj', 'v_proj', 'down_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_confi

TypeError: SFTConfig.__init__() got an unexpected keyword argument 'max_seq_length'

In [10]:
"""
finetune_after_script_b.py

Uses robust_llava_loader.load_model_and_tokenizer() (script B output) to load the model,
auto-detects good LoRA target_modules, and runs PEFT (LoRA) fine-tuning with trl.SFTTrainer.

Usage: python finetune_after_script_b.py
"""

import os
import re
import sys
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel


# try to import helper for preparing k-bit training (peft versions vary)
try:
    from peft import prepare_model_for_kbit_training
except Exception:
    try:
        from peft.utils import prepare_model_for_kbit_training
    except Exception:
        prepare_model_for_kbit_training = None

# Add the src directory to the Python path so we can import robust_llava_loader
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))
from robust_llava_loader import load_model_and_tokenizer

# --------------------- USER CONFIG ---------------------
BASE_MODEL_NAME = "llava-hf/llava-1.5-7b-hf"
DATASET_PATH = "meld_with_rationales.jsonl"   # jsonl containing utterance, sentiment, rationale
OUTPUT_DIR = "./llava-peft-adapters-auto"
USE_4BIT_IF_AVAILABLE = True
MAX_SEQ_LENGTH = 512
PER_DEVICE_BATCH_SIZE = 1  # Reduced batch size further
NUM_EPOCHS = 1
LEARNING_RATE = 2e-4
GRADIENT_ACCUMULATION_STEPS = 4  # Increased accumulation steps further
SAVE_STEPS = 200
LOGGING_STEPS = 20
# -------------------------------------------------------

# helper: create the training prompt
def build_prompt(example):
    return (
        "You are a sentiment analysis expert. Analyze the following utterance and provide "
        "the sentiment along with a step-by-step rationale for your decision.\n\n"
        "### Utterance:\n"
        f"{example.get('utterance','')}\n\n"
        "### Analysis:\n"
        f"Sentiment: {example.get('sentiment','')}\n"
        f"Rationale: {example.get('rationale','')}"
    )

# helper: scan model.named_modules() and choose candidate target module name substrings
def auto_detect_target_module_names(model, prefer_text=True):
    """
    Returns a list of module-name substrings to use in LoraConfig.target_modules.
    Strategy:
      - Collect names of submodules that look like projections (q_proj, k_proj, v_proj, out_proj, o_proj)
      - Prefer modules under 'model' that contain tokens like 'self_attn', 'attn', 'q_proj' etc.
      - If prefer_text=True, try to exclude modules under vision tower (module name containing 'vision' or 'vision_tower')
    """
    proj_patterns = set()
    name_list = [n for n, _ in model.named_modules()]

    for n in name_list:
        # skip top-level empty name
        if not n:
            continue
        # skip vision modules if preferring text modules
        if prefer_text and ("vision" in n or "vision_tower" in n or "vision_model" in n):
            continue
        # find typical projection/fc names in module path
        if re.search(r"(q_proj|k_proj|v_proj|o_proj|out_proj|gate_proj|up_proj|down_proj|fc1|fc2)", n):
            # extract final token (last part after '.')
            final = n.split(".")[-1]
            proj_patterns.add(final)
    # fallback if empty
    if not proj_patterns:
        # default common names
        proj_patterns = {"q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"}
    # keep consistent ordering and return as list
    return sorted(list(proj_patterns))

# UPDATED: tokenize here and enforce max_length/truncation/padding
def format_and_map(example, tokenizer):
    text = build_prompt(example)
    eos = tokenizer.eos_token or ""
    full = text + eos
    # ensure the tokenizer has a pad token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": "<|pad|>"})
    encoded = tokenizer(
        full,
        truncation=True,
        padding="max_length",
        max_length=MAX_SEQ_LENGTH,
        return_attention_mask=True,
    )
    # return input_ids & attention_mask — SFTTrainer can be told to use 'input_ids' as dataset_text_field
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
    }

def main():
    # decide 4-bit usage
    use_4bit = False
    if USE_4BIT_IF_AVAILABLE:
        try:
            import bitsandbytes  # noqa: F401
            use_4bit = True
        except Exception:
            print("bitsandbytes not installed/found — running without 4-bit.")

    # 1) Load model + tokenizer (robust loader)
    print("Loading model + tokenizer (robust loader)...")
    model, tokenizer = load_model_and_tokenizer(model_name=BASE_MODEL_NAME, use_4bit=use_4bit)
    print("Loaded model and tokenizer. Model dtype hint:", getattr(model, "dtype", None))
    model.config.use_cache = False
    try:
        model.gradient_checkpointing_enable()
    except Exception:
        pass

    # 2) Auto-detect target_modules for LoRA (based on model module names)
    print("Auto-detecting candidate LoRA target module name tokens from model.named_modules()...")
    detected = auto_detect_target_module_names(model, prefer_text=True)
    print("Detected target-module name tokens (candidates):", detected)

    # We'll use these tokens as LoraConfig.target_modules (PEFT expects substrings)
    target_modules = detected

    # 3) Prepare model for k-bit training (if using 4-bit & helper present)
    if use_4bit:
        if prepare_model_for_kbit_training is not None:
            print("Preparing model for k-bit training (peft.prepare_model_for_kbit_training)...")
            model = prepare_model_for_kbit_training(model)
        else:
            print("prepare_model_for_kbit_training not available in this peft version — continuing.")

    # 4) Create LoraConfig and wrap model
    lora_cfg = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=target_modules,
    )
    print("Applying LoRA with LoraConfig:", lora_cfg)
    model = get_peft_model(model, lora_cfg)
    print("PEFT/LoRA applied. Peft model keys:", list(model.named_parameters())[:5])

    # 5) Load and format dataset
    print("Loading dataset from", DATASET_PATH)
    ds = load_dataset("json", data_files=DATASET_PATH, split="train")
    print("Dataset size:", len(ds))
    # tokenize & create input_ids & attention_mask fields (and remove other columns)
    ds = ds.map(lambda ex: format_and_map(ex, tokenizer), remove_columns=ds.column_names)

    # Explicitly remove the 'images' column if it exists (it shouldn't in this dataset, but trainer expects it)
    if 'images' in ds.column_names:
        print("Removing 'images' column from dataset...")
        ds = ds.remove_columns("images")

    from trl import SFTTrainer, SFTConfig

    # 6) SFTConfig + trainer
    # NOTE: removed max_seq_length from SFTConfig (it caused your TypeError)
    training_args = SFTConfig(
        output_dir=OUTPUT_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        fp16=use_4bit or (torch.cuda.is_available() and torch.cuda.get_device_properties(0).total_memory >= 12 * 1024 ** 2),
        save_steps=SAVE_STEPS,
        logging_steps=LOGGING_STEPS,
        save_total_limit=3,
        report_to="none",
        dataset_text_field="input_ids",   # <-- point trainer to tokenized input_ids field
        packing=False,
        remove_unused_columns=False, # <-- Add this line
    )


    trainer = SFTTrainer(
        model=model,
        train_dataset=ds,
        peft_config=lora_cfg,
        args=training_args,
    )

       # 7) Dry-run: single step to validate forward/backward
    print("Running a 1-step dry-run to validate training loop...")
    try:
        # Set max_steps in the training_args for the dry run
        trainer.args.max_steps = 1
        trainer.train() # Call train() without the invalid argument
        print("Dry-run succeeded.")
    except Exception as e:
        print("Dry-run failed — inspect traceback. Error:", e)
        # It's good practice to reset the args even if it fails
        trainer.args.max_steps = -1
        raise

    print("Saving adapters to:", OUTPUT_DIR)
    trainer.save_model(OUTPUT_DIR)
    print("Saved. You can load later with PeftModel.from_pretrained(base_model, OUTPUT_DIR)")

if __name__ == "__main__":
    main()

Loading model + tokenizer (robust loader)...
Trying to load LlavaForConditionalGeneration with device_map=auto (quant_config=yes)...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded model and tokenizer. Model dtype hint: torch.float16
Auto-detecting candidate LoRA target module name tokens from model.named_modules()...
Detected target-module name tokens (candidates): ['down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj']
Preparing model for k-bit training (peft.prepare_model_for_kbit_training)...


OutOfMemoryError: CUDA out of memory. Tried to allocate 502.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 224.12 MiB is free. Process 17681 has 14.52 GiB memory in use. Of the allocated memory 13.85 GiB is allocated by PyTorch, and 563.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# (Empty cell placeholder removed for clarity)